In [1]:
# Data processing
import pandas as pd
import numpy as np

# Text preprocessiong
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as Seaborn
import plotly as Plostly
import bokeh as Bokeh

# Dimension reduction
from umap import UMAP

# Clustering
from hdbscan import HDBSCAN

# Count vectorization
from sklearn.feature_extraction.text import CountVectorizer

# Topic model
from bertopic import BERTopic

from collections import Counter



[nltk_data] Downloading package punkt to C:\Users\Mien Trung
[nltk_data]     CPT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Mien Trung
[nltk_data]     CPT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Mien Trung
[nltk_data]     CPT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\Mien Trung CPT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#tải tài liệu lên
import pandas as pd
from sqlalchemy import create_engine

# Thay đổi các thông số kết nối dựa trên cấu hình của bạn
server = 'DESKTOP-CNQUBU5\SLNH'
database = 'booking'
trusted_connection = 'yes'  # Sử dụng Windows Authentication

# Tạo chuỗi kết nối
engine = create_engine(f'mssql+pyodbc://{server}/{database}?trusted_connection={trusted_connection}&driver=ODBC+Driver+17+for+SQL+Server')

# Thay đổi 'ten_bang' thành tên bảng bạn muốn truy vấn
table_name = 'raw.review_en1'

# Thực hiện truy vấn SQL và lấy dữ liệu vào DataFrame
query = f'SELECT * FROM {table_name}'
df = pd.read_sql(query, engine)

# Hiển thị dữ liệu
print(df)

       review_id                                            dislike
0              1  Although the ship is just over a year old, the...
1              6  A jaccuzi bath would be a good feature in the ...
2             22                            Smell is an challenging
3             24  The walls are too thin. I could easily hear pe...
4             25             our room was really cold in the night.
...          ...                                                ...
41942     409042  The water used in our bathroom needs to be str...
41943     409044           The sound-roof between rooms is not good
41944     409046                   The pool could have been cleaned
41945     409049                                          breakfast
41946     409050                  Nothing other than my roommate ha

[41947 rows x 2 columns]


In [3]:
# Get the dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41947 entries, 0 to 41946
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  41947 non-null  int64 
 1   dislike    41947 non-null  object
dtypes: int64(1), object(1)
memory usage: 655.6+ KB


In [4]:
# NLTK English stopwords
stopwords = nltk.corpus.stopwords.words('english')

# Print out the NLTK default stopwords
print(f'There are {len(stopwords)} default stopwords. They are {stopwords}')

There are 179 default stopwords. They are ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'no

In [5]:
# Count vectorizer
vectorizer_model = CountVectorizer(stop_words=stopwords)

In [6]:
print(vectorizer_model)

CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])


In [7]:
# Remove stopwords
df['dislike_without_stopwords'] = df['dislike'].apply(lambda x: ' '.join([w for w in x.split() if w.lower() not in stopwords]))


In [8]:
# Take a look at the data
df.head()

,review_id,dislike,dislike_without_stopwords
0,1,"Although the ship is just over a year old, the...","Although ship year old, hardware showing signs..."
1,6,A jaccuzi bath would be a good feature in the ...,jaccuzi bath would good feature gigantic bathroom
2,22,Smell is an challenging,Smell challenging
3,24,The walls are too thin. I could easily hear pe...,"walls thin. could easily hear people talking, ..."
4,25,our room was really cold in the night.,room really cold night.


In [9]:
# Get the dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41947 entries, 0 to 41946
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   review_id                  41947 non-null  int64 
 1   dislike                    41947 non-null  object
 2   dislike_without_stopwords  41947 non-null  object
dtypes: int64(1), object(2)
memory usage: 983.3+ KB


In [10]:
import string

def remove_special_characters(text):
    cleaned_text = text.translate(str.maketrans('', '', string.punctuation))
    return cleaned_text

In [11]:
df['dislike_remove_special_characters'] = df['dislike_without_stopwords'].apply(remove_special_characters)

In [12]:
def preprocess_dislike(dislike):
    tokens = word_tokenize(dislike.lower()) # Tách từ và chuyển thành chữ thường
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens] # Chuẩn hóa từ
    return lemmatized_tokens

In [13]:
df['dislike_processed'] = df['dislike_remove_special_characters'].apply(preprocess_dislike)

In [14]:
df.head()

,review_id,dislike,dislike_without_stopwords,dislike_remove_special_characters,dislike_processed
0,1,"Although the ship is just over a year old, the...","Although ship year old, hardware showing signs...",Although ship year old hardware showing signs ...,"[although, ship, year, old, hardware, showing,..."
1,6,A jaccuzi bath would be a good feature in the ...,jaccuzi bath would good feature gigantic bathroom,jaccuzi bath would good feature gigantic bathroom,"[jaccuzi, bath, would, good, feature, gigantic..."
2,22,Smell is an challenging,Smell challenging,Smell challenging,"[smell, challenging]"
3,24,The walls are too thin. I could easily hear pe...,"walls thin. could easily hear people talking, ...",walls thin could easily hear people talking la...,"[wall, thin, could, easily, hear, people, talk..."
4,25,our room was really cold in the night.,room really cold night.,room really cold night,"[room, really, cold, night]"


In [15]:
umap_seed = 42 
# Initiate UMAP
umap = UMAP(n_neighbors=15,
            n_components=5,
            min_dist=0.0,
            metric='cosine',
            low_memory=False,
            random_state=umap_seed)  # Đặt hạt giống cố định cho UMAP

# Khởi tạo BERTopic với các tham số cố định
topic_model = BERTopic(umap_model=umap,  # Truyền umap thay vì umap_model
                       vectorizer_model=vectorizer_model,
                       min_topic_size=200,
                       top_n_words=10,
                       language="english",  
                       calculate_probabilities=True)


In [16]:
if hasattr(topic_model, 'umap_model') and hasattr(topic_model.umap_model, 'random_state'):
    topic_model.umap_model.random_state = umap_seed

In [17]:
# Run BERTopic model
topics = topic_model.fit_transform(df['dislike_remove_special_characters'])


In [36]:
# Get the list of topics
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,12880,-1_room_hotel_staff_breakfast,"[room, hotel, staff, breakfast, nothing, one, ...",[room big enough facilities old bathtub high u...
1,0,4775,0_breakfast_food_restaurant_buffet,"[breakfast, food, restaurant, buffet, coffee, ...","[Breakfast buffet, options buffet dinner break..."
2,1,3800,1_noisy_noise_loud_hear,"[noisy, noise, loud, hear, night, music, room,...",[building new hotel next it noisy hear everyth...
3,2,1797,2_smell_room_clean_towels,"[smell, room, clean, towels, bathroom, dirty, ...","[Smell bathroom, smell room, smell room]"
4,3,1725,3_far_location_taxi_town,"[far, location, taxi, town, city, center, walk...","[Location little bit far center, location litt..."
5,4,1670,4_hotel_us_room_staff,"[hotel, us, room, staff, check, asked, pay, on...",[4 stayed room 906 absolutely WIFI Room pretty...
6,5,1402,5_nothing_everything_stay_longer,"[nothing, everything, stay, longer, anything, ...","[Nothing Everything great, Nothing Everything ..."
7,6,1321,6_shower_bathroom_toilet_water,"[shower, bathroom, toilet, water, floor, wet, ...","[Shower, shower room, Shower room]"
8,7,1168,7_pool_swimming_area_bar,"[pool, swimming, area, bar, small, rooftop, su...","[pool, Pool, pool]"
9,8,1114,8_bed_hard_beds_mattress,"[bed, hard, beds, mattress, pillows, comfortab...","[Bed hard, bed hard, bed hard]"


In [19]:
# Get the list of topics
topic_info = topic_model.get_topic_info()

# Display the DataFrame
print(topic_info)

topic_info.to_csv('topic_info.csv', index=False)

    Topic  Count                                   Name  \
0      -1  12880          -1_room_hotel_staff_breakfast   
1       0   4775     0_breakfast_food_restaurant_buffet   
2       1   3800                1_noisy_noise_loud_hear   
3       2   1797              2_smell_room_clean_towels   
4       3   1725               3_far_location_taxi_town   
5       4   1670                  4_hotel_us_room_staff   
6       5   1402       5_nothing_everything_stay_longer   
7       6   1321         6_shower_bathroom_toilet_water   
8       7   1168               7_pool_swimming_area_bar   
9       8   1114               8_bed_hard_beds_mattress   
10      9    983                9_wifi_tv_internet_weak   
11     10    947              10_window_view_light_room   
12     11    846  11_mosquitoes_ants_mosquito_mosquitos   
13     12    738                  12_air_ac_aircon_cold   
14     13    682                13_rooms_room_small_bit   
15     14    583               14_hot_water_shower_cold 

In [20]:
# Hierachical topics
hierarchical_topics = topic_model.hierarchical_topics(df['dislike_remove_special_characters'])

# Take a look at the data
hierarchical_topics

100%|██████████| 30/30 [00:00<00:00, 54.56it/s]


,Parent_ID,Parent_Name,Topics,Child_Left_ID,Child_Left_Name,Child_Right_ID,Child_Right_Name,Distance
29,60,room_breakfast_hotel_bit_could,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",57,nothing_everything_perfect_great_good,59,room_breakfast_hotel_bit_could,1.877628
28,59,room_breakfast_hotel_bit_could,"[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14...",55,room_hotel_bit_water_night,58,breakfast_food_restaurant_staff_options,1.198818
27,58,breakfast_food_restaurant_staff_options,"[0, 17, 19, 23, 27, 30]",41,breakfast_food_restaurant_options_buffet,56,english_staff_speak_nil_communicate,1.145814
26,57,nothing_everything_perfect_great_good,"[5, 18, 20, 21, 22, 24, 25, 29]",53,perfect_everything_nothing_good_great,46,nothing_everything_stay_anything_say,1.139374
25,56,english_staff_speak_nil_communicate,"[17, 19, 23]",40,english_staff_speak_communicate_friendly,23,nil_great_excellent_far_lovely,1.075837
24,55,room_hotel_bit_water_night,"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",39,water_hot_shower_bottles_cold,54,room_hotel_bit_pool_night,1.043659
23,54,room_hotel_bit_pool_night,"[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 15, 1...",52,room_noisy_bed_hotel_night,50,pool_beach_far_location_swimming,1.019014
22,53,perfect_everything_nothing_good_great,"[22, 24, 25, 29]",24,good_fine_everything_everythings_well,44,perfect_everything_nothing_great_ok,0.972199
21,52,room_noisy_bed_hotel_night,"[1, 2, 4, 6, 8, 9, 10, 11, 12, 13, 15]",51,wifi_elevator_tv_lift_stairs,49,room_noisy_bed_night_hotel,0.929035
20,51,wifi_elevator_tv_lift_stairs,"[9, 15]",15,elevator_lift_stairs_floor_luggage,9,wifi_tv_internet_weak_connection,0.861500


In [21]:
import pandas as pd

# Convert hierarchical_topics to DataFrame
hierarchical_topics_df = pd.DataFrame(hierarchical_topics)

# Display the DataFrame
print(hierarchical_topics_df)

   Parent_ID                                 Parent_Name  \
29        60              room_breakfast_hotel_bit_could   
28        59              room_breakfast_hotel_bit_could   
27        58     breakfast_food_restaurant_staff_options   
26        57       nothing_everything_perfect_great_good   
25        56         english_staff_speak_nil_communicate   
24        55                  room_hotel_bit_water_night   
23        54                   room_hotel_bit_pool_night   
22        53       perfect_everything_nothing_good_great   
21        52                  room_noisy_bed_hotel_night   
20        51                wifi_elevator_tv_lift_stairs   
19        50            pool_beach_far_location_swimming   
18        49                  room_noisy_bed_night_hotel   
17        48            pool_beach_far_location_swimming   
16        47                        bed_hard_beds_air_ac   
15        46        nothing_everything_stay_anything_say   
14        45                room_noisy_n

In [22]:
df.to_csv('hierarchical_topics_df.csv', index=False)

In [23]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [24]:
# Topic tree
tree = topic_model.get_topic_tree(hierarchical_topics)

# Print out the tree
print(tree)

.
├─nothing_everything_perfect_great_good
│    ├─perfect_everything_nothing_good_great
│    │    ├─■──good_fine_everything_everythings_well ── Topic: 24
│    │    └─perfect_everything_nothing_great_ok
│    │         ├─■──everything_great_ok_excellent_wonderful ── Topic: 29
│    │         └─perfect_everything_nothing_good_absolutely
│    │              ├─■──perfect_everything_nothing_good_really ── Topic: 25
│    │              └─■──perfect_nothing_everything_note_flawless ── Topic: 22
│    └─nothing_everything_stay_anything_say
│         ├─nothing_everything_stay_anything_say
│         │    ├─nothing_everything_stay_anything_longer
│         │    │    ├─■──nothing_say_really_particular_much ── Topic: 18
│         │    │    └─■──nothing_everything_stay_longer_anything ── Topic: 5
│         │    └─■──hotel_nothing_like_homestay_fault ── Topic: 20
│         └─■──price_fault_value_money_nothing ── Topic: 21
└─room_breakfast_hotel_bit_could
     ├─room_hotel_bit_water_night
     │    ├─wate

In [25]:
# A function to map parent and child topics
def topic_mapping(child_topic_number):
  if child_topic_number in [18,5,20,21]:
    return 'P1_unspecific_complaint'
  elif child_topic_number in [24,29,25,22]:
    return 'P2_Satisfied'
  elif child_topic_number in [30,0,27]:
    return 'P3_food_service'
  elif child_topic_number in [17,19,23]:
    return 'P4_staff_communication_skill'
  elif child_topic_number in [7,16,3,26]:
    return 'P5_beach_poll_location'
  elif child_topic_number in [12,8]:
    return 'P6_bedding_quality'
  elif child_topic_number in [1,4,11]:
    return 'P7_nighttime_noise'
  elif child_topic_number in [13,10]:
    return 'P8_room_view'
  elif child_topic_number in [28,14]:
    return 'P9_Water'
  elif child_topic_number in [2,6]:
    return 'P10_bathroom_quality'
  elif child_topic_number in [15,9]:
    return 'P11_electric_equipment'

In [26]:
# Get the child topic predictions from the basic BERTopic model
child_topic_prediction = topic_model.topics_[:]

# Save the child predictions in the dataframe
df['child_topic'] = child_topic_prediction

# Create the parent topics
df['parent_topic'] = df['child_topic'].apply(topic_mapping)

# Take a look at the data
df.head(10)

,review_id,dislike,dislike_without_stopwords,dislike_remove_special_characters,dislike_processed,child_topic,parent_topic
0,1,"Although the ship is just over a year old, the...","Although ship year old, hardware showing signs...",Although ship year old hardware showing signs ...,"[although, ship, year, old, hardware, showing,...",-1,None
1,6,A jaccuzi bath would be a good feature in the ...,jaccuzi bath would good feature gigantic bathroom,jaccuzi bath would good feature gigantic bathroom,"[jaccuzi, bath, would, good, feature, gigantic...",6,P10_bathroom_quality
2,22,Smell is an challenging,Smell challenging,Smell challenging,"[smell, challenging]",2,P10_bathroom_quality
3,24,The walls are too thin. I could easily hear pe...,"walls thin. could easily hear people talking, ...",walls thin could easily hear people talking la...,"[wall, thin, could, easily, hear, people, talk...",1,P7_nighttime_noise
4,25,our room was really cold in the night.,room really cold night.,room really cold night,"[room, really, cold, night]",12,P6_bedding_quality
5,28,The information provided was not clear and pre...,information provided clear precisely enough ne...,information provided clear precisely enough ne...,"[information, provided, clear, precisely, enou...",4,P7_nighttime_noise
6,29,"Room was noisy, not very good sound proof. Sus...","Room noisy, good sound proof. Sushi restaurant...",Room noisy good sound proof Sushi restaurant e...,"[room, noisy, good, sound, proof, sushi, resta...",1,P7_nighttime_noise
7,83,Breakfast - alittle chaotic when the hotel is ...,"Breakfast - alittle chaotic hotel packed, food...",Breakfast alittle chaotic hotel packed food r...,"[breakfast, alittle, chaotic, hotel, packed, f...",-1,None
8,89,Nothing - a litle paradise,Nothing - litle paradise,Nothing litle paradise,"[nothing, litle, paradise]",-1,None
9,96,The hotel is a 3km short drive from the centre...,hotel 3km short drive centre Hoi however hotel...,hotel 3km short drive centre Hoi however hotel...,"[hotel, 3km, short, drive, centre, hoi, howeve...",3,P5_beach_poll_location


In [27]:
df.to_csv('output.csv', index=False)

In [28]:
df['parent_topic']

0                          None
1          P10_bathroom_quality
2          P10_bathroom_quality
3            P7_nighttime_noise
4            P6_bedding_quality
                  ...          
41942      P10_bathroom_quality
41943        P7_nighttime_noise
41944    P5_beach_poll_location
41945                      None
41946              P8_room_view
Name: parent_topic, Length: 41947, dtype: object

In [29]:
# Lọc giá trị duy nhất của cột 'parent_topic'
unique_parent_topics = df['parent_topic'].drop_duplicates()

# Tạo DataFrame mới từ các giá trị duy nhất của cột 'parent_topic'
unique_parent_topics_df = pd.DataFrame({'parent_topic': unique_parent_topics})

# Thêm cột ID với số thứ tự chạy tự động từ 1
unique_parent_topics_df['ID'] = range(1, len(unique_parent_topics_df) + 1)

# Xuất bảng đã lọc và thiết lập cột ID thành tệp tin CSV
unique_parent_topics_df.to_csv('unique_parent_topics.csv', index=False)

In [30]:
# Take a look at the first review
df['dislike'][0]

'Although the ship is just over a year old, the hardware is showing signs of wear and tear. The shower stall could be cleaner. The room was hot at check in as there was afternoon sun, but thanks to Mai, an electric fan was promptly delivered to our room. The food quality was just above average but the service more than makes up for it.'

In [31]:
# Review frequency by parent topic
df['parent_topic'].value_counts()

parent_topic
P7_nighttime_noise              6316
P3_food_service                 5244
P5_beach_poll_location          3702
P10_bathroom_quality            3118
P1_unspecific_complaint         2585
P6_bedding_quality              1852
P8_room_view                    1629
P11_electric_equipment          1548
P4_staff_communication_skill    1182
P2_Satisfied                    1075
P9_Water                         816
Name: count, dtype: int64

In [42]:
def create_table(root, table, level=1):
    if level not in table:
        table[level] = []

    if 'mapped_topic' in root:
        table[level].append(root['mapped_topic'])

    if 'children' in root:
        for child in root['children']:
            create_table(child, table, level + 1)

# Tạo bảng
table = {}
create_table(tree, table)

# In bảng
print("Level\tTopics")
for level, topics in table.items():
    print(f"{level}\t{', '.join(topics)}")

Level	Topics
1	
2	
3	
4	P2_Satisfied
5	P2_Satisfied, P1_unspecific_complaint, P9_Water, P9_Water, P3_food_service, P3_food_service, P4_staff_communication_skill
6	P2_Satisfied, P2_Satisfied, P1_unspecific_complaint, P5_beach_poll_location, P4_staff_communication_skill, P4_staff_communication_skill
7	P1_unspecific_complaint, P1_unspecific_complaint, P11_electric_equipment, P11_electric_equipment, P5_beach_poll_location
8	P6_bedding_quality, P6_bedding_quality, P5_beach_poll_location, P5_beach_poll_location
9	P8_room_view, P8_room_view, P7_nighttime_noise
10	P7_nighttime_noise, P7_nighttime_noise


In [32]:
# Lấy tất cả các từ trong cột 'dislike' và kết hợp chúng thành một chuỗi
text = ' '.join(df['dislike_remove_special_characters'].dropna())

# Tách các từ
words = text.split()

# Đếm số lần xuất hiện của mỗi từ
word_counts = Counter(words)

# Tính tổng số từ
total_words = sum(word_counts.values())

# Tính xác suất xuất hiện của từng từ
word_probabilities = {word: count/total_words for word, count in word_counts.items()}

# Tạo DataFrame từ từ điển word_probabilities
word_prob_df = pd.DataFrame(list(word_probabilities.items()), columns=['Word', 'Probability'])

# Hiển thị bảng
print(word_prob_df)

            Word  Probability
0       Although     0.000328
1           ship     0.000071
2           year     0.000207
3            old     0.002194
4       hardware     0.000010
...          ...          ...
29980     harass     0.000002
29981        spl     0.000002
29982    smellIn     0.000002
29983      Karst     0.000002
29984  soundroof     0.000002

[29985 rows x 2 columns]
